In [253]:
import numpy as np
import pandas as pd


In [255]:
#Q1
df = pd.read_csv('movie_metadata.csv')
df.drop_duplicates(inplace = True)
df.dropna(subset = ['title_year'], inplace =True)
df = pd.concat([df[['genres','title_year','imdb_score']].groupby(['genres','title_year']).agg(['mean','min','max']),
               df[['genres','title_year', 'duration']].groupby(['genres','title_year']).agg(['sum'])], axis =1)


In [260]:
#Q2

df = pd.read_csv('imdb.csv', escapechar = '\\')
df['year'] = pd.to_numeric(df['year'], errors = 'coerce')
df['title_length'] = df['title'].str.len()

df['quantile'] =  pd.qcut(df['title_length'], q=4, labels = "upto25% upto50% upto75% upto100%".split())
df2 = pd.crosstab(df['year'], df['quantile'])

df3 = df.groupby('year')['title_length'].agg(['min','max','count'])

df4 = pd.merge(df3,df2,on = 'year')

##########    year_vs_tit_len  Trend
year_vs_tit_len = df.groupby('year').mean()['title_length']
#########     rat_brack_vs_tit_len  Trend
df.dropna(inplace = True)
df['rating_brakcet'] = df['imdbRating'].astype(int)
rat_brack_vs_tit_len =  df.groupby('rating_brakcet').mean()['title_length']
df4.tail()

,min,max,count,upto25%,upto50%,upto75%,upto100%
year,,,,,,,
2013.0,9,97,492,109,70,102,211
2014.0,11,106,129,22,29,22,56
2015.0,14,49,8,4,1,1,2
2016.0,16,16,1,1,0,0,0
2017.0,21,21,1,1,0,0,0


In [251]:
#Q3
df = pd.read_csv('diamonds.csv',error_bad_lines = False)
df['z'] = pd.to_numeric(df['z'],errors = 'coerce')
df['Volume'] = df['x'] * df['y'] * df['z']
df['quantile'] =  pd.qcut(df['Volume'], q=4, labels = "0-25% 25-50% 50-75% 75-100%".split())
df1 = pd.crosstab(df['quantile'],df['cut'], normalize = 'all')
df1

cut,Fair,Good,Ideal,Premium,Very Good
quantile,,,,,
0-25%,0.001168,0.015889,0.124182,0.056677,0.052079
25-50%,0.007342,0.024009,0.113373,0.052135,0.053136
50-75%,0.012922,0.032390,0.075699,0.062462,0.066522
75-100%,0.008417,0.018651,0.086267,0.084413,0.052265


In [424]:
#Q4
df = pd.read_csv('movie_metadata.csv')
df.drop_duplicates(inplace = True)
df.dropna(subset = ['title_year'], inplace =True)
# Lat 10 years are 2006 to 2016
df = df[df['title_year']>2006]
df = df[['genres','gross', 'title_year', 'imdb_score']]
df_genres = df['genres'].str.split('|', expand = True).stack().str.get_dummies().sum(level=0)
df = pd.concat([df_genres,df], axis = 1)
df.columns

Index(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Short',
       'Sport', 'Thriller', 'War', 'Western', 'genres', 'gross', 'title_year',
       'imdb_score'],
      dtype='object')

In [421]:
a = pd.DataFrame(df.groupby('title_year')['gross'].quantile(0.9))
a.reset_index(inplace= True)
a

,title_year,gross
0,2007.0,130665433.9
1,2008.0,120136047.0
2,2009.0,120302508.8
3,2010.0,129047302.8
4,2011.0,123207194.0
5,2012.0,148451771.6
6,2013.0,141100034.0
7,2014.0,174218593.7
8,2015.0,177628470.9
9,2016.0,153838924.6


In [425]:
df = df.merge(a, on='title_year', how='left')
df2 = df[df['gross_x']>df['gross_y']]
final_df = pd.concat([df2.groupby('title_year')['imdb_score'].mean(),df2.groupby('title_year')['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Short',
       'Sport', 'Thriller', 'War', 'Western'].sum()], axis = 1)
final_df

,imdb_score,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
title_year,,,,,,,,,,,,,,,,,,,,,
2007.0,6.833333,9,10,4,0,9,1,0,3,4,...,0,2,0,5,3,0,0,4,0,0
2008.0,7.060000,12,9,3,0,5,2,0,8,6,...,1,0,0,5,3,0,0,3,0,0
2009.0,6.804545,10,15,6,1,8,1,0,5,8,...,0,3,0,2,9,0,1,4,1,0
2010.0,6.800000,8,15,6,1,7,0,0,3,9,...,1,0,0,4,5,0,0,1,0,1
2011.0,6.852632,11,13,6,0,8,2,0,2,6,...,1,2,0,1,5,0,1,5,0,0
2012.0,7.322222,7,13,4,1,6,0,0,6,5,...,1,0,0,2,4,0,0,4,1,1
2013.0,7.242105,6,13,4,0,6,3,0,4,6,...,1,0,0,2,7,0,0,3,0,0
2014.0,7.264706,14,15,3,1,6,1,0,4,4,...,0,0,0,1,11,0,0,3,1,0
2015.0,7.235714,7,9,2,0,4,1,0,5,4,...,0,0,0,2,6,0,0,5,0,1


In [526]:
#5
df = pd.read_csv('imdb.csv', escapechar = '\\')
df['year'] = pd.to_numeric(df['year'], errors = 'coerce')
df['decile'] =  pd.qcut(df['duration'], q=10, labels = "0-10% 10-20% 20-30% 30-40% 40-50% 50-60% 60-70% 70-80% 80-90% 90-100%".split())
#df2 = pd.crosstab(df['year'], df['decile'])
df2 = df.groupby('decile')['nrOfNominations','nrOfWins'].sum()
df2

,nrOfNominations,nrOfWins
decile,,
0-10%,7928,3349
10-20%,4923,1912
20-30%,8946,3730
30-40%,3128,2740
40-50%,3109,2511
50-60%,3981,3253
60-70%,5713,4632
70-80%,6470,5473
80-90%,9667,7999


In [527]:

a = df.groupby('decile')['Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport',
       'TalkShow', 'Thriller', 'War', 'Western'].sum()
a.reset_index(inplace=True)
b= a.melt(id_vars=["decile"], 
        var_name="Genre", 
        value_name="Count")
c = b.sort_values(['decile','Count'], ascending=[True, False])
d = c.groupby('decile')['Genre'].head(3)
df3 = pd.DataFrame(d.values.reshape(10,3), index = df2.index)
df3.columns =  ('Top1','Top2','Top3')
final_df = pd.concat([df2,df3],axis =1)
final_df

,nrOfNominations,nrOfWins,Top1,Top2,Top3
decile,,,,,
0-10%,7928,3349,Comedy,Short,Animation
10-20%,4923,1912,Drama,Documentary,Comedy
20-30%,8946,3730,Comedy,Drama,TalkShow
30-40%,3128,2740,Comedy,Drama,Horror
40-50%,3109,2511,Comedy,Drama,Crime
50-60%,3981,3253,Drama,Comedy,Crime
60-70%,5713,4632,Drama,Comedy,Action
70-80%,6470,5473,Drama,Comedy,Action
80-90%,9667,7999,Drama,Comedy,Action


In [497]:

#df.groupby('decile')['duration'].sort()